# Imports

In [ ]:
import math
import matplotlib
import numpy as np
import pandas as pd
import pickle
import sklearn
import tensorflow as tf
import random
import shutil
import os
from enum import Enum
import imblearn
import time

# Learning Model

In [ ]:
def makeCNNModel(evalMetrics, learningRate, inputSize):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = inputSize))
  model.add(tf.keras.layers.AveragePooling2D((3, 3)))
#   model.add(tf.keras.layers.MaxPooling2D((3, 3))) #Test
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(128, activation = 'relu')) #Try to remove
  model.add(tf.keras.layers.Dense(128, activation = 'relu')) #Try to remove
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer = tf.keras.optimizers.Adam(),
                loss = tf.keras.losses.BinaryCrossentropy(),
                metrics = evalMetrics)
  return model

# Load Data

In [ ]:
df = %time pd.read_pickle('/data/AllCircuits.pkl', compression='zip')
dfVal = df.sample(frac=0.2)
df = df.drop(dfVal.index)
labels = df.pop(df.columns.values[-1])
valLabels = dfVal.pop(dfVal.columns.values[-1])
trainHyperImages = np.array(df).reshape(len(df),22,33,33)
valHyperImages = np.array(dfVal).reshape(len(dfVal),22,33,33)

# Traning

In [ ]:
batch_size = 32 # is important to ensure that each batch has a decent chance of containing a few positive samples
numEpochs = 100
learningRate = 0.001 #Eh?Predictor=0.05, default=0.001
dropOut = 0.05 #Eh?Predictor=0.05
evalMetrics = [tf.keras.metrics.TruePositives(name='tp'),
               tf.keras.metrics.FalsePositives(name='fp'),
               tf.keras.metrics.TrueNegatives(name='tn'),
               tf.keras.metrics.FalseNegatives(name='fn'),
               tf.keras.metrics.BinaryAccuracy(name='accuracy'),
               tf.keras.metrics.Precision(name='precision'),
               tf.keras.metrics.Recall(name='recall'),
               tf.keras.metrics.AUC(name='auc')]

inputSize = (22, 33, 33)

model = makeCNNModel(evalMetrics, learningRate, inputSize)

train_history = model.fit(x=trainHyperImages,
                         y=labels,
                         batch_size=batch_size,
                         validation_data=(valHyperImages, valLabels),
                         epochs=numEpochs)

# Generate Train and Test CSVs

In [ ]:
allCircuits = ['/data/CSV/'+x+'/' for x in os.listdir('/data/CSV/')]
# allCircuits = ['/data/CSV/jpeg/', '/data/CSV/aes/', '/data/CSV/swerv/']
allCSVs = list()
for circuit in allCircuits:
  for csv in os.listdir(circuit):
    allCSVs.append(circuit+csv)

DFs = []
for csv in allCSVs:
  DFs.append(pd.read_csv(csv, dtype=np.float32, header=None))
df = pd.concat(DFs)
df.to_pickle('/data/AllCircuits.pkl', compression='zip')